In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_together import Together
from langchain.memory import ConversationBufferMemory
import chainlit as cl

template = """
You are an AI doctor having a concise conversation with a patient. Keep your responses brief and to the point, similar to a real doctor-patient interaction. Ask focused questions, provide short diagnoses, and give concise treatment recommendations. Do not generate or assume any patient responses.

Chat History: {chat_history}
Patient: {human_input}
Doctor:"""

prompt_template = PromptTemplate(
    input_variables=["chat_history", "human_input"],
    template=template
)

together_api_key = "24cdbdf50106e08f6ba3328ac07f97a73eb440ae36da6cdd72f9b091ccca850a"

llm = Together(
    model="MISTRALAI/MIXTRAL-8X7B-INSTRUCT-V0.1",
    temperature=0.7,
    together_api_key=together_api_key
)

conversation_memory = ConversationBufferMemory(
    memory_key="chat_history",
    input_key="human_input",
    return_messages=True
)

llm_chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    memory=conversation_memory
)

@cl.on_chat_start
async def start():
    await cl.Message(content="Hello, I'm an AI doctor. How can I help you today?").send()
    cl.user_session.set("chain", llm_chain)

@cl.on_message
async def main(message: str):
    chain = cl.user_session.get("chain")
    response = await chain.arun(human_input=message)
    await cl.Message(content=response).send()